<a href="https://colab.research.google.com/github/irawan555/MachineLearningTasks/blob/main/13thWeekTasks/BAB%202/Irawan_Mardiansyah_1103210174_Handling_multiple_sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Mochamad Permana Ash Shidiq

NIM : 1103228233

# Handling multiple sequences (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Models expect a batch of inputs

In [ ]:
import torch
# Mengimpor pustaka PyTorch yang digunakan untuk pemrosesan tensor dan deep learning

from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Mengimpor modul dari pustaka HuggingFace Transformers untuk tokenisasi teks dan model klasifikasi urutan

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# Menetapkan nama checkpoint model yang telah dilatih sebelumnya pada dataset SST-2 untuk klasifikasi sentimen

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# Memuat tokenizer yang sesuai dengan checkpoint model yang ditentukan, untuk mengubah teks menjadi token yang bisa diproses oleh model

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
# Memuat model klasifikasi urutan yang sesuai dengan checkpoint yang ditentukan

sequence = "I've been waiting for a HuggingFace course my whole life."
# Menetapkan urutan teks yang akan diklasifikasikan sentimennya

inputs = tokenizer(sequence, return_tensors="pt")
# Menggunakan tokenizer untuk mengubah urutan teks menjadi tensor yang bisa diproses oleh model. Parameter return_tensors="pt" menunjukkan bahwa hasil tokenisasi akan dikembalikan dalam bentuk tensor PyTorch

outputs = model(**inputs)
# Menjalankan model pada input tensor yang telah di-tokenisasi untuk mendapatkan output prediksi

print(outputs)
# Mencetak output dari model yang berisi logits atau skor prediksi untuk setiap kelas sentimen

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [ ]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
# Mengubah 'sequence' menjadi token menggunakan tokenizer
# #tokenizer: Sebuah objek atau fungsi yang bertugas untuk memecah teks menjadi token-token
# #sequence: Teks asli yang akan diubah menjadi token
# #return_tensors="pt": Mengembalikan hasil tokenisasi dalam bentuk tensor PyTorch

print(tokenized_inputs["input_ids"])
# Mencetak token ID dari hasil tokenisasi
# #input_ids: Kunci dalam dictionary 'tokenized_inputs' yang menyimpan ID token hasil tokenisasi

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


In [ ]:
import torch
# Mengimpor pustaka PyTorch untuk operasi tensor

from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Mengimpor AutoTokenizer dan AutoModelForSequenceClassification dari pustaka Transformers oleh HuggingFace

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# Menyimpan nama model pre-trained yang akan digunakan sebagai checkpoint

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# Memuat tokenizer pre-trained sesuai dengan checkpoint yang ditentukan

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
# Memuat model pre-trained untuk tugas klasifikasi urutan sesuai dengan checkpoint yang ditentukan

sequence = "I've been waiting for a HuggingFace course my whole life."
# Mendefinisikan teks input yang akan diklasifikasikan

tokens = tokenizer.tokenize(sequence)
# Tokenisasi teks input menjadi token-token individual

ids = tokenizer.convert_tokens_to_ids(tokens)
# Mengonversi token-token menjadi ID yang sesuai berdasarkan vocabulary model

input_ids = torch.tensor([ids])
# Mengonversi daftar ID token menjadi tensor PyTorch

print("Input IDs:", input_ids)
# Mencetak ID input yang telah dikonversi ke tensor

output = model(input_ids)
# Melakukan inferensi menggunakan model pre-trained dengan input tensor

print("Logits:", output.logits)
# Mencetak hasil logit dari output model

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


Padding the inputs

In [ ]:
batched_ids = [   # Membuat daftar yang berisi beberapa daftar ID
    [200, 200, 200],  # Daftar pertama yang berisi tiga ID dengan nilai 200
    [200, 200]        # Daftar kedua yang berisi dua ID dengan nilai 200
]

In [ ]:
padding_id = 100
# #padding_id: ID yang digunakan untuk mengisi nilai padding dalam batch

batched_ids = [
    [200, 200, 200],
    # #baris_1: Batch pertama dengan semua ID bernilai 200
    [200, 200, padding_id],
    # #baris_2: Batch kedua dengan dua ID bernilai 200 dan satu ID sebagai padding
]

In [ ]:
# Memuat model pra-latih untuk klasifikasi urutan dari checkpoint yang diberikan
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Mendefinisikan ID token untuk urutan pertama
sequence1_ids = [[200, 200, 200]]
# Mendefinisikan ID token untuk urutan kedua
sequence2_ids = [[200, 200]]
# Mendefinisikan ID token untuk batch yang berisi dua urutan dengan padding pada urutan kedua
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

# Mencetak logits dari model untuk urutan pertama
print(model(torch.tensor(sequence1_ids)).logits)
# Mencetak logits dari model untuk urutan kedua
print(model(torch.tensor(sequence2_ids)).logits)
# Mencetak logits dari model untuk batch yang berisi dua urutan
print(model(torch.tensor(batched_ids)).logits)

tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3373, -1.2163]], grad_fn=<AddmmBackward0>)


Attention masks

In [ ]:
# Membuat batch ID yang terdiri dari dua sekuens
# Batch pertama: [200, 200, 200]
# Batch kedua: [200, 200, tokenizer.pad_token_id] (dengan padding)
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

# Membuat attention mask untuk dua sekuens
# Batch pertama: [1, 1, 1] (semua token diaktifkan)
# Batch kedua: [1, 1, 0] (token terakhir adalah padding dan tidak diaktifkan)
attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

# Mengirim input ke model untuk mendapatkan output
# torch.tensor digunakan untuk mengubah batched_ids dan attention_mask menjadi tensor
# output logits dihasilkan oleh model
outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))

# Menampilkan hasil output logits dari model
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
